# Denotation AllenNLP SRL BiLSTM

In [16]:
from allennlp_models.pretrained import load_predictor

In [17]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [18]:
from checklist.pred_wrapper import PredictorWrapper

In [19]:
import logging
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

In [20]:
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("John cooks pasta and Anna eats it.")
output

{'verbs': [{'verb': 'cooks',
   'description': '[ARG0: John] [V: cooks] [ARG2: pasta] and Anna eats it .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'O', 'O', 'O', 'O', 'O']},
  {'verb': 'eats',
   'description': '[ARG0: John cooks pasta and Anna] [V: eats] [ARG1: it] .',
   'tags': ['B-ARG0',
    'I-ARG0',
    'I-ARG0',
    'I-ARG0',
    'I-ARG0',
    'B-V',
    'B-ARG1',
    'O']}],
 'words': ['John', 'cooks', 'pasta', 'and', 'Anna', 'eats', 'it', '.']}

In [21]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [22]:
d = ["John cooks pasta and Anna eats it."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'cooks',
     'description': '[ARG0: John] [V: cooks] [ARG2: pasta] and Anna eats it .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG2', 'O', 'O', 'O', 'O', 'O']},
    {'verb': 'eats',
     'description': '[ARG0: John cooks pasta and Anna] [V: eats] [ARG1: it] .',
     'tags': ['B-ARG0',
      'I-ARG0',
      'I-ARG0',
      'I-ARG0',
      'I-ARG0',
      'B-V',
      'B-ARG1',
      'O']}],
   'words': ['John', 'cooks', 'pasta', 'and', 'Anna', 'eats', 'it', '.']}],
 array([1.]))

In [23]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description']

In [24]:
def get_arg(pred, arg_target='ARG1'):
    # we assume one predicate:
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [25]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description']
        
    return predicate_structure

In [26]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [39]:
object = ['it']

In [41]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    object = set([meta['object']])
    arg_0 = get_arg(pred, arg_target='ARG1')

    if arg_0 == object:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [42]:
# initialize editor object
editor = Editor()

# create examples
t = editor.template("{first_name} {last_name} cooks pasta and {first_name} {last_name} eats {object}.", object = object, meta=True, remove_duplicates = True, nsamples=1000)

In [44]:
with open('/datasets/denotation_bilstm.txt', 'w') as f:
    print(t.data, file = f)

# initialize a test object

import csv
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('/results/denotation_bilstm.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp) 

Predicting 1000 examples
Test cases:      1000
Fails (rate):    0 (0.0%)
['Henry', 'Harris', 'cooks', 'pasta', 'and', 'Henry', 'Harris', 'eats', 'it', '.'] [ True]
['Elaine', 'Robinson', 'cooks', 'pasta', 'and', 'Elaine', 'Robinson', 'eats', 'it', '.'] [ True]
['Albert', 'Parker', 'cooks', 'pasta', 'and', 'Albert', 'Parker', 'eats', 'it', '.'] [ True]
['Joseph', 'Walker', 'cooks', 'pasta', 'and', 'Joseph', 'Walker', 'eats', 'it', '.'] [ True]
['Larry', 'Thomas', 'cooks', 'pasta', 'and', 'Larry', 'Thomas', 'eats', 'it', '.'] [ True]
['Emma', 'Bennett', 'cooks', 'pasta', 'and', 'Emma', 'Bennett', 'eats', 'it', '.'] [ True]
['Joan', 'Gray', 'cooks', 'pasta', 'and', 'Joan', 'Gray', 'eats', 'it', '.'] [ True]
['Francis', 'Turner', 'cooks', 'pasta', 'and', 'Francis', 'Turner', 'eats', 'it', '.'] [ True]
['Heather', 'James', 'cooks', 'pasta', 'and', 'Heather', 'James', 'eats', 'it', '.'] [ True]
['Elizabeth', 'Ward', 'cooks', 'pasta', 'and', 'Elizabeth', 'Ward', 'eats', 'it', '.'] [ True]
['P

# End of Notebook